# 🎯 조별 프로젝트: E2E MLOps Pipeline

## 개요

3일간 학습한 MLOps 기술을 종합하여 **완전한 E2E ML 파이프라인**을 구축합니다.

### 평가 기준

| 항목 | 배점 | 기준 |
|------|------|------|
| Kubeflow Pipeline | 40점 | 최소 5개 컴포넌트, Succeeded 상태 |
| MLflow Tracking | 20점 | 최소 2회 Run, 파라미터/메트릭 기록 |
| Feature Engineering | 10점 | 1개 이상 파생 피처 생성 |
| KServe 배포 (선택) | 25점 | InferenceService 생성 및 API 테스트 |

---

## 1. 환경 설정

In [ ]:
# 패키지 설치
!pip install kfp>=2.0.0 mlflow>=2.9.0 scikit-learn pandas numpy -q
print("✅ 패키지 설치 완료!")

In [ ]:
# Import
import os
from kfp import dsl
from kfp.dsl import component, Input, Output, Dataset
from kfp import compiler
import kfp

print(f"KFP Version: {kfp.__version__}")

In [ ]:
# ⚠️ TODO: 팀 설정으로 변경하세요!
TEAM_NAME = "team-XX"                        # 예: team-01, team-02, ...
USER_NAMESPACE = "kubeflow-user-example-com" # 현재 Kubeflow 프로필 네임스페이스

MLFLOW_TRACKING_URI = "http://mlflow-server-service.mlflow-system.svc.cluster.local:5000"

print(f"Team: {TEAM_NAME}")
print(f"Namespace: {USER_NAMESPACE}")
print(f"MLflow URI: {MLFLOW_TRACKING_URI}")

---

## 2. 컴포넌트 구현

### 2.1 데이터 로드

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "scikit-learn==1.3.2"]
)
def load_data(data_source: str, output_data: Output[Dataset]):
    """California Housing 데이터셋 로드"""
    import pandas as pd
    from sklearn.datasets import fetch_california_housing
    
    print("=" * 60)
    print("  Step 1: Load Data")
    print("=" * 60)
    
    housing = fetch_california_housing(as_frame=True)
    df = housing.frame
    
    print(f"  Source: {data_source}")
    print(f"  Shape: {df.shape}")
    
    df.to_csv(output_data.path, index=False)
    print(f"  ✅ Data saved")

### 2.2 전처리

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "scikit-learn==1.3.2", "numpy==1.24.3"]
)
def preprocess(
    input_data: Input[Dataset],
    X_train_out: Output[Dataset],
    X_test_out: Output[Dataset],
    y_train_out: Output[Dataset],
    y_test_out: Output[Dataset],
    test_size: float = 0.2
):
    """데이터 전처리"""
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    
    print("=" * 60)
    print("  Step 2: Preprocess")
    print("=" * 60)
    
    df = pd.read_csv(input_data.path)
    
    X = df.drop(columns=['MedHouseVal'])
    y = df['MedHouseVal']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42
    )
    
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    
    X_train_scaled.to_csv(X_train_out.path, index=False)
    X_test_scaled.to_csv(X_test_out.path, index=False)
    y_train.to_csv(y_train_out.path, index=False)
    y_test.to_csv(y_test_out.path, index=False)
    
    print(f"  Train: {len(X_train)}, Test: {len(X_test)}")
    print(f"  ✅ Preprocessing completed")

### 2.3 피처 엔지니어링 ⭐ (평가 항목)

**TODO**: 팀에서 창의적인 피처를 추가하세요!

#### 피처 아이디어

```python
# 1. 방당 침실 비율
df['bedroom_ratio'] = df['AveBedrms'] / df['AveRooms']

# 2. 가구당 인구
df['people_per_household'] = df['Population'] / df['AveOccup']

# 3. Bay Area까지 거리
df['dist_to_bay'] = np.sqrt((df['Latitude']-37.77)**2 + (df['Longitude']+122.42)**2)

# 4. 소득 구간
df['income_category'] = pd.cut(df['MedInc'], bins=5, labels=[1,2,3,4,5])

# 5. 밀집도
df['density'] = df['Population'] * df['AveOccup']
```

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "numpy==1.24.3"]
)
def feature_engineering(
    X_train_in: Input[Dataset],
    X_test_in: Input[Dataset],
    X_train_out: Output[Dataset],
    X_test_out: Output[Dataset]
) -> int:
    """
    피처 엔지니어링
    
    ⭐ TODO: 팀에서 창의적인 피처를 추가하세요!
    """
    import pandas as pd
    import numpy as np
    
    print("=" * 60)
    print("  Step 3: Feature Engineering")
    print("=" * 60)
    
    X_train = pd.read_csv(X_train_in.path)
    X_test = pd.read_csv(X_test_in.path)
    
    original_cols = list(X_train.columns)
    
    def add_features(df):
        """⭐ TODO: 여기에 새로운 피처를 추가하세요!"""
        df = df.copy()
        
        # 예시 1: 방당 침실 비율
        df['bedroom_ratio'] = df['AveBedrms'] / (df['AveRooms'] + 1e-6)
        
        # 예시 2: 가구당 인구
        df['people_per_household'] = df['Population'] / (df['AveOccup'] + 1e-6)
        
        # TODO: 더 많은 피처 추가!
        
        return df
    
    X_train_fe = add_features(X_train)
    X_test_fe = add_features(X_test)
    
    new_cols = [c for c in X_train_fe.columns if c not in original_cols]
    print(f"  New features: {new_cols}")
    print(f"  Total features: {len(X_train_fe.columns)}")
    
    X_train_fe.to_csv(X_train_out.path, index=False)
    X_test_fe.to_csv(X_test_out.path, index=False)
    
    print(f"  ✅ Feature engineering completed")
    return len(new_cols)

### 2.4 모델 학습 (MLflow 연동)

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "scikit-learn==1.3.2", "mlflow==2.9.2", "numpy==1.24.3"]
)
def train_model(
    X_train: Input[Dataset],
    X_test: Input[Dataset],
    y_train: Input[Dataset],
    y_test: Input[Dataset],
    mlflow_tracking_uri: str,
    experiment_name: str,
    team_name: str,
    n_estimators: int = 100,
    max_depth: int = 10
) -> str:
    """모델 학습 및 MLflow 기록"""
    import pandas as pd
    import numpy as np
    import mlflow
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    import os
    
    print("=" * 60)
    print(f"  Step 4: Train Model - {team_name}")
    print("=" * 60)
    
    X_train_df = pd.read_csv(X_train.path)
    X_test_df = pd.read_csv(X_test.path)
    y_train_df = pd.read_csv(y_train.path)
    y_test_df = pd.read_csv(y_test.path)
    
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment(experiment_name)
    
    with mlflow.start_run(run_name=f"{team_name}-run") as run:
        run_id = run.info.run_id
        
        mlflow.log_params({"n_estimators": n_estimators, "max_depth": max_depth})
        mlflow.set_tag("team", team_name)
        
        model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train_df, y_train_df.values.ravel())
        
        y_pred = model.predict(X_test_df)
        r2 = r2_score(y_test_df, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_df, y_pred))
        mae = mean_absolute_error(y_test_df, y_pred)
        
        mlflow.log_metrics({"r2": r2, "rmse": rmse, "mae": mae})
        
        print(f"  R2: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")
        print(f"  ✅ Run ID: {run_id}")
    
    return run_id

### 2.5 모델 평가 및 배포

In [ ]:
@component(base_image="python:3.9-slim", packages_to_install=["mlflow==2.9.2"])
def evaluate_model(run_id: str, mlflow_tracking_uri: str, r2_threshold: float = 0.75) -> str:
    """모델 평가 및 배포 결정"""
    import mlflow
    import os
    
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    
    client = mlflow.tracking.MlflowClient()
    run = client.get_run(run_id)
    r2 = float(run.data.metrics.get("r2", 0))
    
    decision = "deploy" if r2 >= r2_threshold else "skip"
    print(f"  R2: {r2:.4f}, Threshold: {r2_threshold}, Decision: {decision}")
    
    return decision


@component(base_image="python:3.9-slim", packages_to_install=["kubernetes==28.1.0"])
def deploy_model(run_id: str, model_name: str, namespace: str):
    """KServe 배포 (namespace는 현재 프로필과 동일해야 함)"""
    from kubernetes import client, config
    from kubernetes.client.rest import ApiException
    import time
    
    print(f"  Deploying {model_name} to {namespace}")
    
    try:
        config.load_incluster_config()
    except:
        config.load_kube_config()
    
    api = client.CustomObjectsApi()
    isvc = {
        "apiVersion": "serving.kserve.io/v1beta1",
        "kind": "InferenceService",
        "metadata": {"name": model_name, "namespace": namespace},
        "spec": {"predictor": {"sklearn": {"storageUri": f"mlflow-artifacts:/{run_id}/model"}}}
    }
    
    try:
        api.delete_namespaced_custom_object("serving.kserve.io", "v1beta1", namespace, "inferenceservices", model_name)
        time.sleep(5)
    except ApiException:
        pass
    
    api.create_namespaced_custom_object("serving.kserve.io", "v1beta1", namespace, "inferenceservices", isvc)
    print(f"  ✅ InferenceService created")


@component(base_image="python:3.9-slim")
def send_alert(run_id: str, team_name: str):
    """성능 미달 알림"""
    print(f"  ⚠️ Alert: {team_name} model did not meet threshold")

---

## 3. 파이프라인 정의

In [ ]:
@dsl.pipeline(name="Team Project Pipeline", description="E2E ML Pipeline")
def project_pipeline(
    data_source: str = "sklearn",
    team_name: str = "team-XX",
    experiment_name: str = "team-XX-experiment",
    model_name: str = "team-XX-model",
    namespace: str = "kubeflow-user-example-com",
    mlflow_tracking_uri: str = "http://mlflow-server-service.mlflow-system.svc.cluster.local:5000",
    n_estimators: int = 100,
    max_depth: int = 10,
    r2_threshold: float = 0.75
):
    load_task = load_data(data_source=data_source)
    
    preprocess_task = preprocess(input_data=load_task.outputs["output_data"])
    
    feature_task = feature_engineering(
        X_train_in=preprocess_task.outputs["X_train_out"],
        X_test_in=preprocess_task.outputs["X_test_out"]
    )
    
    train_task = train_model(
        X_train=feature_task.outputs["X_train_out"],
        X_test=feature_task.outputs["X_test_out"],
        y_train=preprocess_task.outputs["y_train_out"],
        y_test=preprocess_task.outputs["y_test_out"],
        mlflow_tracking_uri=mlflow_tracking_uri,
        experiment_name=experiment_name,
        team_name=team_name,
        n_estimators=n_estimators,
        max_depth=max_depth
    )
    
    evaluate_task = evaluate_model(
        run_id=train_task.output,
        mlflow_tracking_uri=mlflow_tracking_uri,
        r2_threshold=r2_threshold
    )
    
    with dsl.If(evaluate_task.output == "deploy"):
        deploy_model(run_id=train_task.output, model_name=model_name, namespace=namespace)
    
    with dsl.If(evaluate_task.output == "skip"):
        send_alert(run_id=train_task.output, team_name=team_name)

print("✅ 파이프라인 정의 완료!")

---

## 4. 컴파일 및 실행

In [ ]:
# 컴파일
pipeline_file = f'{TEAM_NAME}_pipeline.yaml'

compiler.Compiler().compile(
    pipeline_func=project_pipeline,
    package_path=pipeline_file
)

print(f"✅ Pipeline compiled: {pipeline_file}")
print(f"")
print(f"📋 Next Steps:")
print(f"  1. Kubeflow UI → Pipelines → Upload pipeline")
print(f"  2. Select {pipeline_file}")
print(f"  3. Create Run → Set parameters")
print(f"")
print(f"⚠️  Important:")
print(f"  namespace 파라미터를 현재 Kubeflow 프로필과 동일하게 설정하세요!")

---

## 5. 트러블슈팅

### S3 권한 오류
```
S3UploadFailedError: AccessDenied
```
→ 본 코드는 S3 artifact 저장을 비활성화했으므로 발생하지 않음

### RBAC 권한 오류
```
Forbidden: cannot create/delete inferenceservices
```
→ `namespace` 파라미터를 현재 Kubeflow 프로필의 네임스페이스와 동일하게 설정

### 컴포넌트 데이터 전달 오류
```python
# ❌ 잘못된 방법
train_task = train_model(X_train=preprocess_task)

# ✅ 올바른 방법
train_task = train_model(X_train=preprocess_task.outputs["X_train_out"])
```

---

## 6. 발표 체크리스트

### 필수 확인 사항
- [ ] 파이프라인 컴파일 성공
- [ ] 모든 컴포넌트 Succeeded
- [ ] MLflow에 Run 기록 확인
- [ ] Feature Engineering에서 새 피처 추가

### 발표 내용 (15분)
1. 팀 소개 (1분)
2. 아키텍처 (2분)
3. 구현 하이라이트 (4분) - Feature Engineering 중심
4. 데모 (4분) - Kubeflow UI, MLflow UI
5. 트러블슈팅 (1분)
6. Q&A (3분)